In [1]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import sys
import json


#New imports
from textblob import TextBlob
import pandas as pd
import numpy as np
from textblob import TextBlob

# Configuration

In [2]:
TWEET_COUNT = 2

## Tokens for the Tweepy API

In [3]:
ckey="hpbARu5lJlozST82JEdXYcaS7"
csecret="JpfbXATmdNPz6yIhFwmVU1mGU28QTICCuYkfqGibb25HV4x4VK"
atoken="1094363214043889665-UohVaHgV89P7OtSgUlWq6QJ6q30Yu1"
asecret="mz0RhOmUHN2ISAFcE3j0bOwZVxoAy5x5Ccn8RAiqYrRRI"

# Tweet Retrieval

In [13]:
class tweetFetcher:
    
    
    def __init__(self, ckey, csecret, atoken, asecret):
        
        auth = OAuthHandler(ckey, csecret)
        auth.set_access_token(atoken, asecret)
        self.api = tweepy.API(auth)        
        self.actor_list = []
    
    
    #Manage API limit
    def limit_handled(self, cursor):
        while True:
            try:
                yield cursor.next()
            except tweepy.RateLimitError:
                time.sleep(15*60)
            except StopIteration:
                break
    

### FUNCTIONS THAT HANDLE ACTOR LIST ###
    
    def create_actor_list_from_file(self, input_actor_file):
        with open(input_actor_file) as actor_file:
            actor_list = []
            for line in actor_file:
                actor_list.append(line.strip())
        self.actor_list = actor_list
        
        
    def set_actor_list(self, provided_actor_list):
        self.actor_list = provided_actor_list
        
    def get_actor_list(self):
        return self.actor_list

    
    # assumes actor_list has no duplicates
    def run_query(self, output_file):
    
        with open(output_file, "w") as write_file:
            for actor in self.actor_list:
                tweet_list = []
                
                
                
                cursor = tweepy.Cursor(self.api.search, q=actor, lang='en', tweet_mode='extended').items(TWEET_COUNT)
                
        
                for tweet in self.limit_handled(cursor):
                    
                    #Get the text of the tweet
                    
                    if hasattr(tweet, 'text'):
                        contents = tweet.text
                    else:
                        contents = tweet.full_text
                    
                    
                    datetime = tweet.created_at
                    tweet_tuple = datetime, contents
                    tweet_list.append(tweet_tuple)
                    
                    json.dump(tweet_tuple, write_file, indent=4, sort_keys=True, default=str)


In [14]:
#Example of how to use TweetFetcher


def fetch_tweets():
    tweet_fetcher = tweetFetcher(ckey, csecret, atoken, asecret)
    
    tweet_fetcher.create_actor_list_from_file("intermediates/test_actors.txt")
    
    tweet_fetcher.run_query("intermediates/data_file.json")

fetch_tweets()

# Sentiment Analysis

In [8]:

class sentiment_analyzer:
    
    
    #Given a tweet, return the sentiment
    def get_tweet_sentiment(self, tweet):
        
        sentiment = TextBlob(tweet).sentiment
        
        return sentiment
    
    
    def mass_tweet_analyis(self, archive_file):
        
        
        with open(archive_file) as json_file:  
            data = json.load(json_file)
            for p in data:
                print(p)
        
        
        
    
    ##Deprecated Method: Do not USE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1111111111111111111111111
    
    def update_tweet_sentiment(self, actor_tweet_dictionary):
    
        tweets_by_actor = actor_tweet_dictionary
        for actors in tweets_by_actor:
            for tweets in (tweets_by_actor[actors]):
                tweets.update(sentiment=TextBlob(tweets['content']).sentiment)
                print(tweets)
            
        return(tweets_by_actor)

In [9]:
sentiment_analyzer = sentiment_analyzer()

In [10]:
sentiment_analyzer.get_tweet_sentiment("i am a bad dumb stupid idiot loser")

Sentiment(polarity=-0.66875, subjectivity=0.7416666666666667)